Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pieplines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

In [9]:
# Import Statements
from sklearn.pipeline import Pipeline
# from sklearn.datasets import fetch_20newsgroups
# from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [7]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
rfc = RandomForestClassifier()

In [8]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])

In [9]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [10]:
grid_search.best_score_

0.8926487747957993

In [11]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1], dtype=int64)

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [2]:
train.head()

,id,description,category
0,1,A marriage of 13 and 18 year old bourbons. A m...,2
1,2,There have been some legendary Bowmores from t...,1
2,3,This bottling celebrates master distiller Park...,2
3,4,What impresses me most is how this whisky evol...,1
4,9,"A caramel-laden fruit bouquet, followed by une...",2


In [3]:
train.description[1]

"There have been some legendary Bowmores from the mid-60s and this is every bit their equal. All of them share a remarkable aroma of tropical fruit, which here moves into hallucinatory intensity: guava, mango, peach, pineapple, grapefruit. There’s a very light touch of peat smoke, more a memory of Islay than the reality. Concentrated; even at low strength the palate is silky, heady, and haunting, and lasts forever in the dry glass. A legend is born. (Eight bottles only for the U.S.) Editor's Choice."

In [4]:
train.description[2]

"This bottling celebrates master distiller Parker Beam's 50 years of service by including whiskey from each of the past five decades. This is a fabulous whiskey: seamless and incredibly complex, with an impeccable marriage of youth and maturity. It’s also very even-keeled throughout -- quite different than last year’s equally impressive PHC, a 27 year old, whose personality was more like an exhilarating old wooden rollercoaster ride (and also brandished more oak).\xa0Look for candied citrus, nectarine, blueberry, and sultana anchored by a nougat center, laced with honeyed vanilla and orange creamsicle. There’s a dusting of cocoa powder, brittle mint, and cinnamon, too. Tobacco leaves, polished leather, and teasing bourbon barrel char round out the palate, emerging more prominently towards a warming finish. A classic!"

### Define Pipeline Components

In [13]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
rfc = RandomForestClassifier()

In [14]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])

In [17]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train.description, train.category)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   53.3s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

## Submission 5

In [91]:
from lightgbm import LGBMClassifier

vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
lgb = LGBMClassifier(random_state=42)

In [92]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('lgb', lgb)
                ])

In [128]:
params ={
    'lgb__num_leaves': (15, 18, 20, 22, 25, 30, 35), 
    'lgb__min_child_samples': (25, 50, 75, 100), 
    'lgb__min_child_weight': [1e-5],
    'vect__max_df': [0.75],
    'vect__min_df': [.02],
    'vect__max_features': [1000]
}

In [129]:
from sklearn.model_selection import RandomizedSearchCV
search = RandomizedSearchCV(pipe, params, cv=10, n_iter=10, n_jobs=-1, verbose=5)
search.fit(train.description, train.category)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.2min finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'lgb__num_leaves': (15, 18, 20, 22, 25, 30, 35), 'lgb__min_child_samples': (25, 50, 75, 100), 'lgb__min_child_weight': [1e-05], 'vect__max_df': [0.75], 'vect__min_df': [0.02], 'vect__max_features': [1000]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=5)

In [130]:
search.best_score_

0.9122196442382057

In [131]:
search.best_params_

{'vect__min_df': 0.02,
 'vect__max_features': 1000,
 'vect__max_df': 0.75,
 'lgb__num_leaves': 35,
 'lgb__min_child_weight': 1e-05,
 'lgb__min_child_samples': 50}

## Stacked submission

In [3]:
import pandas as pd
# Filenames of your submissions you want to ensemble
files = ['data/submission5.csv', 'data/submission2.csv']
 
target = 'category'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]
 
sample_submission = pd.read_csv('data/submission1.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)

In [7]:
submission.category = submission.category.astype(int)

In [8]:
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [9]:
submission.to_csv('data/ensemble-submission2.csv', index=False)

## Submission 6

In [ ]:
params = {'vect__min_df': [0.02],
 'vect__max_features': [1000],
 'vect__max_df': [0.75],
 'lgb__num_leaves': [35],
 'lgb__min_child_weight': [1e-05],
 'lgb__min_child_samples': [50]}

In [12]:
# import spacy
# import string
# nlp = spacy.load("en_core_web_lg")
# stopwords = nlp.Defaults.stop_words
# punctuation = list(string.punctuation)

# def get_chunks(text):
#     doc = nlp(text)
#     chunks = list(doc.noun_chunks)
#     chunks = [str(i) for i in chunks]
#     return [i.lower() for i in chunks if i not in stopwords and i not in punctuation]

In [16]:
def get_noun_chunks(text):
    doc = nlp(text)
    chunks = []
    for ent in doc.ents:
        chunks.append(ent.text)
    return chunks

In [ ]:
train['transformed'] = train.description.apply(get_noun_chunks)

In [18]:
from lightgbm import LGBMClassifier

vect = TfidfVectorizer(tokenizer=get_noun_chunks, stop_words='english')
lgb = LGBMClassifier(random_state=42)

pipe = Pipeline([('vect', vect), ('lgb', lgb)])

from sklearn.model_selection import RandomizedSearchCV
search = RandomizedSearchCV(pipe, params, cv=5, n_iter=2, n_jobs=1, verbose=5)
search.fit(train.description, train.category)

search.best_score_

C:\Users\Owen\Anaconda3\envs\nlp1\lib\site-packages\sklearn\model_selection\_search.py:271: UserWarning: The total space of parameters 1 is smaller than n_iter=2. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] vect__min_df=0.02, vect__max_features=1000, vect__max_df=0.75, lgb__num_leaves=35, lgb__min_child_weight=1e-05, lgb__min_child_samples=50 
[CV]  vect__min_df=0.02, vect__max_features=1000, vect__max_df=0.75, lgb__num_leaves=35, lgb__min_child_weight=1e-05, lgb__min_child_samples=50, score=0.637065637065637, total= 2.1min
[CV] vect__min_df=0.02, vect__max_features=1000, vect__max_df=0.75, lgb__num_leaves=35, lgb__min_child_weight=1e-05, lgb__min_child_samples=50 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.7min remaining:    0.0s


[CV]  vect__min_df=0.02, vect__max_features=1000, vect__max_df=0.75, lgb__num_leaves=35, lgb__min_child_weight=1e-05, lgb__min_child_samples=50, score=0.6332046332046332, total= 1.9min
[CV] vect__min_df=0.02, vect__max_features=1000, vect__max_df=0.75, lgb__num_leaves=35, lgb__min_child_weight=1e-05, lgb__min_child_samples=50 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  6.3min remaining:    0.0s


...

[CV]  vect__min_df=0.02, vect__max_features=1000, vect__max_df=0.75, lgb__num_leaves=35, lgb__min_child_weight=1e-05, lgb__min_child_samples=50, score=0.632495164410058, total=  59.2s
[CV] vect__min_df=0.02, vect__max_features=1000, vect__max_df=0.75, lgb__num_leaves=35, lgb__min_child_weight=1e-05, lgb__min_child_samples=50 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  9.7min remaining:    0.0s


[CV]  vect__min_df=0.02, vect__max_features=1000, vect__max_df=0.75, lgb__num_leaves=35, lgb__min_child_weight=1e-05, lgb__min_child_samples=50, score=0.6337209302325582, total=  50.8s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 11.2min finished


0.6337973704563031

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [132]:
# Predictions on test sample
pred = search.predict(test['description'])

In [133]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [134]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [135]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission5.csv', index=False)

## Challenge

You're trying to achienve 90% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [37]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, 
                   algorithm='randomized',
                   n_iter=10)

In [38]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators': (5, 10, 20)
}

In [44]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [46]:
# # Fit
# grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=4, verbose=1)
# grid_search.fit(data.data, data.target)

In [14]:
grid_search.best_score_

0.8786464410735122

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [68]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
lgb = LGBMClassifier(random_state=42)

lsi = Pipeline([('vect', vect), ('svd', svd)])
pipe = Pipeline([('lsi', lsi), ('lgb', lgb)])

params ={
    'lgb__num_leaves': (6, 15, 30, 40, 50), 
    'lsi__vect__max_df': ( 0.75, 1.0),
    'lsi__vect__min_df': (.02, .05),
    'lsi__vect__max_features': (500,1000),
    'lsi__svd__n_components': [10,100],
}

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [69]:
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=-1, verbose=10)
grid_search.fit(train.description, train.category)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   46.5s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  6

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'lgb__num_leaves': (6, 15, 30, 40, 50), 'lsi__vect__max_df': (0.75, 1.0), 'lsi__vect__min_df': (0.02, 0.05), 'lsi__vect__max_features': (500, 1000), 'lsi__svd__n_components': [10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [70]:
grid_search.best_params_

{'lgb__num_leaves': 15,
 'lsi__svd__n_components': 100,
 'lsi__vect__max_df': 1.0,
 'lsi__vect__max_features': 500,
 'lsi__vect__min_df': 0.02}

In [71]:
grid_search.best_score_

0.9071925754060325

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [72]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [73]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [74]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [75]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission4.csv', index=False)

## submission 6

In [85]:
from xgboost import XGBClassifier
vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
xgb = XGBClassifier(random_state=42)

lsi = Pipeline([('vect', vect), ('svd', svd)])
pipe = Pipeline([('lsi', lsi), ('xgb', xgb)])

params ={ 
    'lsi__vect__max_df': [0.9, 1.0],
    'lsi__svd__n_components': [10,100],
    'xbg__max_depth': (5, 10, 15, 20)
}

In [86]:
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=-1, verbose=5)
grid_search.fit(train.description, train.category)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   

ValueError: Invalid parameter xbg for estimator Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1))]). Check the list of available parameters with `estimator.get_params().keys()`.

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [19]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [20]:
doc = nlp("Two bananas in pyjamas")

In [21]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [26]:
len(X) == len(data.data)

True

In [22]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [23]:
X = get_word_vectors(data.data)

rfc.fit(X, data.target)

/Users/jonathansokoll/anaconda3/envs/U4-S1-NLP-DS6/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
rfc.score(X, data.target)

0.9918319719953326

## Follow Along

In [53]:
import spacy
nlp = spacy.load("en_core_web_lg")

def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [54]:
X = get_word_vectors(train.description)

rfc.fit(X, train.category)

C:\Users\Owen\Anaconda3\envs\nlp1\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [57]:
rfc.score(X, train.category)

0.9899458623356535

In [63]:
pred = rfc.predict(get_word_vectors(test['description']))
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [64]:
submission.to_csv('./data/submission3.csv', index=False)

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on